<a href="https://colab.research.google.com/github/JacopoBartoli/vas_regression/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Install packages and organize imports.
In this section we install the needed packages and import them.
We set some variables for the used paths, and mount GDrive.

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorboard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time, datetime, math, io, sklearn.preprocessing, itertools
from sklearn.metrics import confusion_matrix

Save some useful paths.

In [ ]:
# Path to the datasets.
DATASET_DIR = '/content/gdrive/My Drive/IVA/data/'
# Path to where we save logs.
LOGS_DIR = '/content/gdrive/My Drive/IVA/logs'
# Path to where we save the checkpoints.
CHECKPOINT_DIR = '/content/gdrive/My Drive/IVA/checkpoint/train'
# Path to where we save the model.
MODEL_DIR = '/content/gdrive/My Drive/IVA/model'

Mount the drive.

In [ ]:
# Mount your drive to access the dataset.
from google.colab import drive
drive.mount('/content/gdrive')

#2) Define our transformer.
In this section we implement our transformer model.

##2.1) Utility functions.
Define some utilities functions, for the positional encodings and the feed forward network.

In [ ]:
# Define the positional encoding function.
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
# Define the feed forward network
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
                              tf.keras.layers.Dense(dff, activation='relu'),
                              tf.keras.layers.Dense(d_model)
  ])

##2.2) Define the encoder layer.


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = tf.keras.layers.MultiHeadAttention(num_heads, output_shape=d_model, key_dim=24)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    attn_output = self.mha(x,x,x,mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

##2.3) Define the encoder.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1):
    super(Encoder, self).__init__()
    
    self.d_model = d_model
    self.num_layers = num_layers

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate ) for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x

## 2.4) Define the transformer with the LSTM  layer.

In this section we define our model using the layers defined above. Only the encoder part of a transformed model is used. The output of this encoder is feeded to a LSTM that perform regression.

In [ ]:
class EncoderRegressor(tf.keras.Model):
  def __init__(self, feat_dim, max_len, d_model, n_heads, num_layers, dim_feedforward, num_classes=1, dropout=0.1, pos_encoding='fixed', activation='sigmoid'):

    super(EncoderRegressor, self).__init__()

    self.max_len = max_len
    self.d_model = d_model
    self.n_heads = n_heads
    
    self.flatten_inp = tf.keras.layers.Flatten()

    self.project_inp = tf.keras.layers.Dense(max_len*d_model)

    self.reshape = tf.keras.layers.Reshape((max_len, d_model))

    self.pos_encoding = positional_encoding(2048, self.d_model)


    self.encoder_layer = Encoder(num_layers = num_layers, d_model = d_model, num_heads = n_heads, dff= dim_feedforward, rate=0.01)

    self.act = tf.keras.activations.get(activation)
    self.dropout = tf.keras.layers.Dropout(dropout)

    self.lstm = tf.keras.layers.LSTM(num_classes, activation=self.act)

    self.feat_dim = feat_dim
    self.num_classes = num_classes

  def call(self, inputs, training):    
      enc_padding_mask = None
      seq_len = tf.shape(inputs)[1]

      # Flatten the input tensor and map in a different vector space(d_model)
      x = self.flatten_inp(inputs)
      x = self.project_inp(x)

      # Reshape the tensor to adapt the shape [batch_size, sequence_lenght, d_model]
      x = self.reshape(x)

      # Positional encoding.
      x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
      x += self.pos_encoding[:, :seq_len, :]

      
      # Encoder Layer
      x = self.encoder_layer(x, training, enc_padding_mask)
      x = self.dropout(x, training=training)

      # LSTM
      x = self.lstm(x)

      return x


  

#3) Manage the data.
In this section we manipulate and extract the data.

##3.1) Load the train set.

Define the name of the dataset used for 
training.

The data in the .csv can have a variable number of features. But three column are always needed. They are 'Sequenza', 'Frame' and 'Label'.
The first represent the id of a sequence, the second the id of a frame. The third represent the label of each frame.


Each row of the file need to represent a frame, and each frame of the same sequence need to have the same label.



In [ ]:
# Name of the dataset used.
TRAIN_SET = 'train-velocity-66-sampled.csv'

Load the train set from a .csv file.

In [ ]:
df = pd.read_csv(DATASET_DIR + TRAIN_SET)
print(df.head())

## 3.2) Divide the data in train and validation set.

The data is splitted in train and validation set. After that we divide the label from the features.


In [ ]:
# Dimension of the validation set.
VAL_PERCENTAGE = 10
valid_dim = math.floor(df['Sequenza'].max()*VAL_PERCENTAGE / 100)
train_dim = df['Sequenza'].max() - valid_dim
max_seq = df['Sequenza'].max()

# Needed later for confusion matrix
number_of_labels = df['Label'].tolist()
number_of_labels = len(list(dict.fromkeys(number_of_labels))) - 1


# Divide by train and validation set.
df_valid = df.loc[df['Sequenza'] >= (max_seq - valid_dim)]
df_train = df.loc[df['Sequenza'] < train_dim]
df_valid = df_valid.drop(['Frame'], axis=1)
df_train = df_train.drop(['Frame'], axis=1)

# Extract the labels.
lbl_valid = df_valid['Label']
lbl_train = df_train['Label']


# Remove the labels from the data.
df_valid = df_valid.drop(['Label'], axis = 1)
df_train = df_train.drop(['Label'], axis = 1)

In [ ]:
print(df_train.head())

##3.3) Preprocessing of the sequences.

In the dataset each row represent a frame of the sequence. Each frame in a sequence has the same label. We want to make some preprocessing for having a dataset that has a single label for each sequence (not one for each frame). We want that each item of the dataset represent a whole sequence and not a frame.


In [ ]:
# Need to pass different in a separate ways lbl and data.
def preprocessing_sequences(data, lbl):
  # Preprocess the labels.
  # The label and the ids of the sequence are concatenated together.
  seq_ids = data['Sequenza']

  tmp = pd.concat([seq_ids, lbl], axis=1)
  tmp = tmp.set_index('Sequenza')
  # Then they are gourped by sequence id so we can have a single label for each
  # sequence.
  tmp = tmp.groupby(level='Sequenza').mean()

  labels = tmp['Label'].values

  min_seq = data['Sequenza'].min()
  num_seqs = data['Sequenza'].max() - data['Sequenza'].min() + 1
  min_seq = int(min_seq)
  num_seqs = int(num_seqs)

  # Create the new dataset.
  temp = []
  for id in tqdm(range(min_seq, min_seq + num_seqs)):
    # Extract sequences one by one.
    seq = data.loc[data['Sequenza'] == id]

    # Remove the unused columns.
    seq = seq.drop(['Sequenza'], axis=1)
    num_col = len(seq.columns)

    # Iterate over each row of the selected sequence  
    temp_row = []
    for index, row in seq.iterrows():
      temp_row = np.append(temp_row, row)
    temp_row = np.reshape(temp_row, (-1, num_col))

    temp.append(temp_row[:])

  return temp, labels


In [ ]:
df_train, lbl_train = preprocessing_sequences(df_train, lbl_train)
df_valid, lbl_valid = preprocessing_sequences(df_valid, lbl_valid)

## 3.3) Create and manage the train and validation set.


In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((df_train, lbl_train))
ds_valid = tf.data.Dataset.from_tensor_slices((df_valid, lbl_valid))

In [ ]:
BATCH_SIZE = 8
BUFFER_SIZE = 5000
random_seed = 1337

Function to apply some preprocessing when making batches.

In [ ]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE,seed=random_seed)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE))

Now we divide in batches the validation and training sets.

In [ ]:
train_batches = make_batches(ds_train)
val_batches = make_batches(ds_valid)

#4) Training Phase.

In this section we organize all the operation needed to perform the train of the model and evaluate its performance.

## 4.1)Set the hyperparameters.
Set the transformer hyperparameter, define the learning rate, optimizer and loss type.

In [ ]:
# Model hyperparameters
d_model = 32 # Dimension of the hidden representation.
dim_feedforward = 256
n_heads = 6
num_layers = 3
feat_dim = len(df.columns) - 3 # Number of feature inside each item of the sequence.

# Changed.
max_len=  len(df_train[0]) # Length of each sequence.

# Parameter needed for separate classification from regression.
# For now just regression is implemented.
num_classes = 1

# Network hyperparameter
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optim_name = 'Adam'
# tfa.optimizers.RectifiedAdam() seems to not work properly.

# This loss and accuracy objects are meant for regression.
# For classifications other metrics will be needed.
loss_object = tf.keras.losses.MeanSquaredError()
error_object = tf.keras.metrics.MeanAbsoluteError()

## 4.2) Custom implementation of the loss and accuracy functions.

Add a way to customize the loss and accuracy functions.

In [ ]:
def loss_function(real,pred):

  loss_ = loss_object(real, pred)

  return loss_

def error_function(real, pred):

  accuracies = error_object(real, pred)
  
  return accuracies

Create the metric objects.

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_error = tf.keras.metrics.Mean(name='train_error')
val_error = tf.keras.metrics.Mean(name='validion_error')
val_loss = tf.keras.metrics.Mean(name='validation_loss')

## 4.4) Manage checkpoint and Tensorboard.
Create the model and load last checkpoint if it exist.

In [ ]:
# Create the transformer.
transformer = EncoderRegressor(d_model=d_model, dim_feedforward=dim_feedforward, n_heads=n_heads, num_layers=num_layers, feat_dim=feat_dim, max_len=max_len, num_classes = num_classes)

# Checkpoint management.
use_checkpoint = False
if use_checkpoint:
  ckpt = tf.train.Checkpoint(trasformer=transformer,
                           optimizer=optimizer)

  ckpt_manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=5)
  if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Restored latest checkpoint")

Set paths for tensorboard visualization.

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = LOGS_DIR + '/gradient_tape/' + current_time + '/train'
valid_log_dir = LOGS_DIR + '/gradient_tape/' + current_time + '/valid'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

### 4.4.1) Informations needed for the confusion matrix visualization inside tensorboard.

In [ ]:
# Needed for confusion matrix visualization
# This can be done just because the regression task has a finite number of labels.
tag_list = [float(_dummy) for _dummy in range(number_of_labels + 1)]

Utility function needed for the confusion matrix visualization.

In [ ]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  # Remove number equals to zero.
  divider = cm.sum(axis=1)[:, np.newaxis]
  divider = np.where(divider!=0, divider, 1)

  labels = np.around(cm.astype('float') / divider, decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

## 4.5) Train the model.
Set the number of epoch and define the train step.



In [ ]:
# Number of epochs
EPOCHS = 30

In [ ]:
def valid_step(inp,tar):
  tar_real = tar

  
  predictions = transformer(inp, training = False) 
  loss = loss_function(tar_real, predictions)
  accuracy = error_function(tar_real, predictions)


  val_loss(loss)
  val_error(accuracy)
  
  return predictions


In [ ]:
def train_step(inp,tar):
  tar_real = tar

  with tf.GradientTape() as tape:
    predictions = transformer(inp, training = True) 
    loss = loss_function(tar_real, predictions)
    accuracy = error_function(tar_real, predictions)
    
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))


    train_loss(loss)
    train_error(accuracy)
  
  return predictions

Start the training.

The summary writers will save the prediction and ground truth distributions.
They will save the loss and the error too.

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()
  # Needed for histogram visualization.
  predictions_histogram = []
  labels_histogram = []
  y_pred, y_true = [],[]
  
  train_loss.reset_states()
  train_error.reset_states()

  for (batch, (inp, tar)) in enumerate(train_batches):
    predictions = train_step(inp, tar)
    # Save the histogram of predictions.
    predictions_histogram = np.hstack((predictions_histogram, tf.reshape(predictions, len(predictions))))    
    labels_histogram = np.hstack((labels_histogram, tar))
    # Data needed for confusion matrix
    y_pred.extend(np.around(np.array(predictions)*number_of_labels))
    y_true.extend(np.around(np.array(tar)*number_of_labels))

  # Build confusion matrix.
  cm = confusion_matrix(y_pred, y_true)
  figure = plot_confusion_matrix(cm, class_names=tag_list)
  cm_image = plot_to_image(figure)

  with train_summary_writer.as_default():
    tf.summary.scalar('Loss', train_loss.result(), step=epoch)
    tf.summary.scalar('Error', train_error.result(), step=epoch)
    tf.summary.histogram('Predictions distribution', predictions_histogram, step = epoch)
    tf.summary.histogram('Ground Truth distribution', labels_histogram, step = epoch)
    tf.summary.image('Confusion Matrix', cm_image, step=epoch)
  
  if epoch == EPOCHS - 1:
    with train_summary_writer.as_default():
      tf.summary.histogram('Predictions distribution last epoch', predictions_histogram, step = 0)
      tf.summary.histogram('Ground Truth distribution last epoch', labels_histogram, step = 0)
  
  predictions_histogram = []
  labels_histogram = []
  val_loss.reset_states()
  val_error.reset_states()

  for (batch, (inp,tar)) in enumerate(val_batches):
    predictions = valid_step(inp,tar)
    # Save the histogram of predictions.
    predictions_histogram = np.hstack((predictions_histogram, tf.reshape(predictions, len(predictions))))    
    labels_histogram = np.hstack((labels_histogram, tar))

  with valid_summary_writer.as_default():
    tf.summary.scalar('Loss', val_loss.result(), step=epoch)
    tf.summary.scalar('Error', val_error.result(), step=epoch)
    tf.summary.histogram('Predictions distribution', predictions_histogram, step = epoch)
    tf.summary.histogram('Ground Truth distribution', labels_histogram, step = 0)
  
  if epoch == EPOCHS - 1:
    with valid_summary_writer.as_default():
      tf.summary.histogram('Predictions distribution last epoch', predictions_histogram, step = 0)
      tf.summary.histogram('Ground Truth distribution last epoch', labels_histogram, step = 0)



  if (epoch + 1) % 5 == 0 and use_checkpoint:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Error {train_error.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

if use_checkpoint:
  print(ckpt_manager.save())

## 4.6) Save the model.
Save the summary of the model on tensorboard.

In [ ]:
hyperparameters = [
                   f'Dimension of the encoded representation: {d_model}',
                   f'Dimension of the feedforward layers: {dim_feedforward}',
                   f'Number of attention heads: {n_heads}',
                   f'Number of encoding layers: {num_layers}',
                   f'Number of starting features: {feat_dim}',
                   f'Max_length of each sequence: {max_len}',
                   f'Learning rate: {learning_rate}',
                   f'Optimizer: {optim_name}'
                   
]

In [ ]:
def get_summary_str(model):
    lines = []
    model.summary(print_fn=lines.append)
    for item in hyperparameters:
      lines.append(item)
    # Add initial spaces to avoid markdown formatting in TensorBoard
    return '    ' + '\n    '.join(lines)

# Add the summary as text in Tensorboard
with train_summary_writer.as_default():
  tf.summary.text('Model configuration', get_summary_str(transformer), step=0)

Save the transformer model in .h5 format.

In [ ]:
transformer.save(MODEL_DIR + '/' + current_time + '/transformers')